In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Dependencies

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## Model Instantiation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## Encode and Calculate Sentiment

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
token = tokenizer.encode('This is an awesome car', return_tensors= 'pt')

In [ ]:
result= model(token)

In [ ]:
result.logits

tensor([[-2.0514, -2.4098, -0.5984,  1.2760,  3.0748]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

5

## Collecting reviews


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

data = pd.read_excel('/content/drive/MyDrive/MBFS/MBINDIA_profile_scraping.xlsx')
column_data = data['latestComments/0/text']
reviews = []

for data in column_data:
    if pd.notnull(data):
        comments = data.strip()
        reviews.append(comments)

In [ ]:
reviews

['.',
 '❤️❤️❤️❤️❤️ أنا بحب مرسيدس ❤️',
 'Umma',
 'Price',
 '❤️❤️❤️❤️❤️❤️❤️❤️❤️',
 '❤️',
 'Navratri yaad nhi the kya...??',
 'Koenigsegg',
 'Mere paas isse acha model hai aur mehngi bhi',
 'Gift me 1 or 2',
 '❤️❤️❤️❤️',
 'Big+',
 'Where are climate activists?',
 '❤️❤️❤️❤️',
 'Wow',
 'Something missing in ad',
 'Just a ❤️❤️',
 'The only failure in Mercedes is this “Mercedes Me App”…I got fed up after raise a multiple issues with this App. They did multiple updates and all still this app is not working… waste to promote this its purely failure..',
 '✔️',
 'Beautiful',
 'Pls support Pro golf n India..rather than glorifying 18 H’cappers who shoot a 14 over to win these tournaments..',
 'Free offers for you  \n \nThis is carzzyypremium\n(carzzyypremium) an established Luxury car detailing and service  brand with over 12 years experience.\n we  are giving you following offers\nGet 20% on any Periodic Service \n50% off on Ceramic Coating at \n50% off Paint Protection Film and more   ️\nDon’t m

## load reviews and score them

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df

,review
0,.
1,❤️❤️❤️❤️❤️ أنا بحب مرسيدس ❤️
2,Umma
3,Price
4,❤️❤️❤️❤️❤️❤️❤️❤️❤️
...,...
1469,Happinese
1470,price of this car
1471,So you are telling me.. In a zombie apocalypse...
1472,When is it coming to India


In [ ]:
df['review'].iloc[0]

'.'

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,.,3
1,❤️❤️❤️❤️❤️ أنا بحب مرسيدس ❤️,5
2,Umma,3
3,Price,4
4,❤️❤️❤️❤️❤️❤️❤️❤️❤️,5
...,...,...
1469,Happinese,3
1470,price of this car,5
1471,So you are telling me.. In a zombie apocalypse...,1
1472,When is it coming to India,1
